In [1]:
import tensorflow
from tensorflow.keras.models import Sequential
from keras.utils import np_utils
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import regularizers, optimizers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
import numpy as np

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255
y_train_cat = to_categorical(y_train, 10)
y_test_cat = to_categorical(y_test, 10)
print("x_train.shape:", x_train.shape)
print("y_train_cat.shape", y_train.shape)


170508288/170498071 [==============================] - 6s 0us/step
x_train.shape: (50000, 32, 32, 3)
y_train_cat.shape (50000, 1)


In [3]:
num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)

Callbacks included to improve performance

In [4]:
rls = ReduceLROnPlateau(monitor='val_accuracy', mode ='max', factor=0.5, min_lr=1e-7, verbose = 1, patience=10)
es = EarlyStopping(monitor='val_accuracy', mode='max', verbose = 1, patience=50)
mc = ModelCheckpoint('cnn_best_model.h5', monitor='val_accuracy', mode='max', verbose = 1, save_best_only=True)
callback_list = [rls, es, mc]

In [5]:
def build_model(lr = 0, dc = 0, dr = 0):
    model = Sequential(name = 'CNN_cifar10')
    model.add(Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_uniform', kernel_regularizer=regularizers.l2(1e-4), padding='same', input_shape=(32, 32, 3)))
    model.add(BatchNormalization())
    model.add(Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_uniform', kernel_regularizer=regularizers.l2(1e-4), padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_uniform', kernel_regularizer=regularizers.l2(1e-4), padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_uniform', kernel_regularizer=regularizers.l2(1e-4), padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.3))

    model.add(Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_uniform', kernel_regularizer=regularizers.l2(1e-4), padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_uniform', kernel_regularizer=regularizers.l2(1e-4), padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.4))

    model.add(Flatten())
    model.add(Dense(256, activation='elu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(Dropout(dr))
    model.add(Dense(10, activation='softmax'))
    # compile model
    opt = optimizers.Adam(lr = lr, decay = dc)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

In [6]:
model = build_model(lr = 0.001, dc = 1e-5, dr = 0.5)
model.summary()



Model: "CNN_cifar10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 batch_normalization_1 (Batc  (None, 32, 32, 32)       128       
 hNormalization)                                                 
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 32)       0         
 )                                                               
                                                       

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [7]:
#data augmentation
datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False
    )
datagen.fit(x_train)

In [8]:
model.fit_generator(datagen.flow(x_train, y_train_cat, batch_size = 64),
                                 validation_data = (x_test, y_test_cat),
                                 steps_per_epoch = x_train.shape[0] // 64, 
                                 epochs = 20, 
                                 verbose = 1,
                                 callbacks = callback_list)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/20
781/781 [==============================] - ETA: 0s - loss: 1.8209 - accuracy: 0.4123
Epoch 00001: val_accuracy improved from -inf to 0.49320, saving model to cnn_best_model.h5
781/781 [==============================] - 60s 62ms/step - loss: 1.8209 - accuracy: 0.4123 - val_loss: 1.6502 - val_accuracy: 0.4932 - lr: 0.0010
Epoch 2/20
781/781 [==============================] - ETA: 0s - loss: 1.3166 - accuracy: 0.5651
Epoch 00002: val_accuracy improved from 0.49320 to 0.61050, saving model to cnn_best_model.h5
781/781 [==============================] - 57s 73ms/step - loss: 1.3166 - accuracy: 0.5651 - val_loss: 1.2881 - val_accuracy: 0.6105 - lr: 0.0010
Epoch 3/20
781/781 [==============================] - ETA: 0s - loss: 1.1524 - accuracy: 0.6280
Epoch 00003: val_accuracy improved from 0.61050 to 0.66950, saving model to cnn_best_model.h5
781/781 [==============================] - 48s 62ms/step - loss: 1.1524 - accuracy: 0.6280 - val_loss: 1.0602 - val_accuracy: 0.6695 - lr: 0.

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(test_acc) #test_accuracy found to be above 90% for certain conditions when necessary


OSError: ignored

In [ ]:
import pathlib
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url, untar=True)
data_dir = pathlib.Path(data_dir)

In [26]:
sunflower_url = "https://static.toiimg.com/thumb/msid-67586673,width-1070,height-580,overlay-toi_sw,pt-32,y_pad-40,resizemode-75,imgsize-3918697/67586673.jpg"
#sunflower_path = tensorflow.keras.utils.get_file('Red_sunflower', origin=sunflower_url)
sunflower_path = tensorflow.keras.utils.get_file(origin=sunflower_url)  # took away the 'Red_sunflower', 

img_height = 32
img_width = 32

class_names = y_train

img = tensorflow.keras.utils.load_img(
    sunflower_path, target_size=(img_height, img_width)
)
img_array = tensorflow.keras.utils.img_to_array(img)
img_array = tensorflow.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tensorflow.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

This image most likely belongs to [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.] with a 23.20 percent confidence.
